# Explore Concepts needed for Lighting Controller

In [1]:
%matplotlib inline

import numpy as np
from IPython.display import display
import time
from pprint import pp

from model_helper import *
from display_helper import *

## Light Profiles

In [ ]:
m = Model()
print("Scenarios:")
display( m.MODEL_SCENARIO_CONFIG)

print("Color Profiles:")
display( m.COLOR_PROFILE)

print(f"LED Patterns:{m.BRIGHTNESS_PATTERN_FUNCTION.keys()}" )

## Model Configuration

In [ ]:
for c in m.MODEL_CONFIG:
    print(f"\n'{c}': (rows,cols)=({m.MODEL_CONFIG[c]['leds']['rows']},{m.MODEL_CONFIG[c]['leds']['cols']}) => LED ID array shape: {m.MODEL_CONFIG[c]['led_ids'].shape}")
    print(m.MODEL_CONFIG[c]['led_ids'])

## Load/Generate a Scenario

In [2]:
# Create a model
m = Model()

# Initialize the Light Scenario
SCENARIO = 'Idle'
m.init_model_scenario(SCENARIO)
cp = m.MODEL_SCENARIO_CONFIG[SCENARIO]['color_profile']
pat = m.MODEL_SCENARIO_CONFIG[SCENARIO]['led_pattern']
rgb = m.COLOR_PROFILE[cp]['rgb']
hls = rgb_tuple_to_hls( rgb )

print(f"Scenario '{SCENARIO}':")
print(f"=> LED Pattern: '{pat}', Color Profile '{cp}' => rgb: {rgb} => hls: ({hls[0]:.3f},{hls[1]:.3f},{hls[2]:.3f}))")
print(f"=> Brightness Scale: {m.MODEL_SCENARIO_CONFIG[SCENARIO]['brightness_scale']}, Cycle Time: {m.MODEL_SCENARIO_CONFIG[SCENARIO]['cycle_time']}")

# Timesteps per Cycle = Cycle Time [secs/cycle] / LED timestep time [secs/timesteps]
t_cycle = float(m.MODEL_SCENARIO_CONFIG[m.scenario]['cycle_time']) / m.LED_TIMESTEP_SEC
print(f"=> Timesteps per Cycle: {t_cycle}")

Scenario 'Idle':
=> LED Pattern: 'ellipse', Color Profile '40W Tungsten' => rgb: (255, 197, 143) => hls: (0.080,0.780,1.000))
=> Brightness Scale: 0.5, Cycle Time: 4.0
=> Timesteps per Cycle: 80.0


## Animate the Scenario

In [3]:
# Draw the sides and top of the model
m.draw_model_sides_top()

# Draw the LEDs for each color profile and then add them together as a GIF
# Baseline Fastest: Avg LED Update Time: 22.0 ms over 80 timesteps => Total Pattern: 1758.2 ms
# Pre-allocated LED Array: Average LED Update Time: 20.9 ms over 80 timesteps => Total Pattern: 1675.9 ms
# Removed vectorize() for brightness to rgb: Average LED Update Time: 25.2 ms over 80 timesteps => Total Pattern: 2014.1 ms
# Using nditer for brightness to rgb: Average LED Update Time: 13.9 ms over 80 timesteps => Total Pattern: 1114.9 ms
# Using file: Average LED Update Time: 6.4 ms over 80 timesteps => Total Pattern: 511.1 ms
images = []
upd_time_sum = 0.0
upd_time_count = 0
for t in range(2*m.N_LED_PATTERN_TIMESTEPS):
    start_time = time.time()
    m.update_led_pattern( timestep=t, distance=(0.5,None), proximity={'Entrance':False, 'Exit':False} )
    stop_time = time.time()
    upd_time_sum += (stop_time - start_time)
    upd_time_count += 1

    # Append this snapshot of the model at timestep t to a list of images,
    # which will be used to build an animated GIF image.
    images.append( m.model.copy() )

print(f"Average LED Update Time: {1000.0 * (upd_time_sum / upd_time_count):0.1f} ms over { upd_time_count:,d} timesteps => Total Pattern: { 1000.0*upd_time_sum:0.1f} ms")


Average LED Update Time: 6.4 ms over 160 timesteps => Total Pattern: 1022.7 ms


In [ ]:
# Display the GIF
display_images( images, ms=50, loop=0 )

In [ ]:
# # Compare performance of LED Update Pattern method
# # Fastest Run: 20.4 ms ± 206 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
# # Fastest using nditer: 11.8 ms ± 272 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# # Fastest using file: 6.37 ms ± 263 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# %timeit m.update_led_pattern( timestep=0 )

In [ ]:
# m.update_led_pattern( timestep=10 )
# m.model

# Generate All Scenario files

In [4]:
for s in m.MODEL_SCENARIO_CONFIG:
    print(f"Scenario '{s}': Generating LED Pattern Buffer file... ", end='')
    m.init_model_scenario(s)
    print(f" completed.")

Scenario 'Idle': Generating LED Pattern Buffer file...  completed.
Scenario 'Standard': Generating LED Pattern Buffer file...  completed.
Scenario 'Energy': Generating LED Pattern Buffer file...  completed.
Scenario 'diag_calibrate_distance': Generating LED Pattern Buffer file...  completed.
Scenario 'diag_come_in': Generating LED Pattern Buffer file...  completed.
Scenario 'diag_ellipse': Generating LED Pattern Buffer file...  completed.
Scenario 'diag_range': Generating LED Pattern Buffer file...  completed.
Scenario 'diag_all_on': Generating LED Pattern Buffer file...  completed.
Scenario 'diag_all_off': Generating LED Pattern Buffer file...  completed.
Scenario 'diag_cp_candle': Generating LED Pattern Buffer file...  completed.
Scenario 'diag_cp_40w_tungsten': Generating LED Pattern Buffer file...  completed.
Scenario 'diag_cp_100w_tungsten': Generating LED Pattern Buffer file...  completed.
Scenario 'diag_cp_halogen': Generating LED Pattern Buffer file...  completed.
Scenario 'dia